# Microestructuras y sistemas de Trading
## Laboratorio 1

## Equipo C
### Participantes:
>Paulina Milenka Glez Manrique, Andres García Ruvalcaba, Mariana López Correa,Juan López Pelayo.

In [9]:
import quandl 
import pandas as pd
import os
import calendar
import numpy as np

In [10]:
quandl.ApiConfig.api_key = "6ARMqMT_1jbzJ71XyRCx"

In [11]:
path=os.getcwd()
files=os.listdir(path)
etf_files = [f for f in files if f[-3:]=='csv']

In [12]:
df=[]
for f in etf_files: 
    data=pd.read_csv(f, header=0, sep=',', index_col=0, parse_dates=False, skip_blank_lines=True, skiprows=10)

    df.append(data)

In [13]:
df_nassets = []
for f in etf_files:
    data=pd.read_csv(f,nrows=4)
    data=data.iloc[3,0]
    df_nassets.append(data)

In [14]:
df_fechas=[]
for f in etf_files:
    data=pd.read_csv(f,nrows=4)
    data=data.iloc[1,0]
    df_fechas.append(data)

In [15]:
dic = {'abr':'04','ago':'08','dic':'12','feb':'02','ene':'01','jun':'06','jul':'07','mar':'03','may':'05','nov':'11','oct':'10','sep':'09'}
lenfechas=len(df_fechas)
for i in range(lenfechas):
    df_fechas[i]=df_fechas[i].replace(df_fechas[i][3:6],dic.get(df_fechas[i][3:6]))

In [17]:
l=len(df_fechas)
for i in range(l):
    df_fechas[i]=df_fechas[i][-4:]+df_fechas[i][2:6]+df_fechas[i][0:2]
df_fechas

['2017-02-28',
 '2017-03-31',
 '2017-04-28',
 '2017-05-31',
 '2017-06-30',
 '2017-07-31',
 '2017-08-31',
 '2017-09-29',
 '2017-10-31',
 '2017-11-30',
 '2017-12-29',
 '2018-01-31']

In [18]:
M1= df[0]
tickers = M1.index.get_values()
tickers = tickers.tolist()
prices = quandl.get_table('WIKI/PRICES',ticker = tickers,  qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                       date = { 'gte': df_fechas[0], 'lte': df_fechas[-1] }, 
                       paginate=True)
prices.head()

,ticker,date,adj_close
None,,,
0,ZTS,2018-01-31,76.73
1,ZTS,2018-01-30,78.35
2,ZTS,2018-01-29,79.18
3,ZTS,2018-01-26,80.09
4,ZTS,2018-01-25,79.25


In [ ]:
p_prices=prices.pivot('ticker','date','adj_close')
#p_prices=p_prices.fillna(p_prices.mean(axis=1))
p_prices=p_prices.fillna(method='ffill',axis=1)

In [ ]:
q_acc=[]
l=len(df)
for i in range(l):
    data=df[i]['Acciones'].dropna()
    data=data.sort_index(ascending=True)
    q_acc.append(data)
q_acc[0].head()

In [ ]:
acciones=[]
l=len(q_acc)
for j in range(l):
    lista=pd.DataFrame()
    l=len(q_acc[j])
    for i in range(l):
        data=q_acc[j].to_frame()
        data=data.iloc[i].replace(',','')
        lista=lista.append(data)
    acciones.append(lista)

In [ ]:
l=len(acciones)
acciones2 = []
for i in range(l):
    a=acciones[i]
    a['Acciones']=a['Acciones'].str.replace(',','')
    #a['Acciones']=a['Acciones'].str.replace('.00','')
    #a['Acciones']=a['Acciones'].astype(int)
    acciones2.append(a)

In [ ]:
a=acciones[0]
a['Acciones']=a['Acciones'].str.replace(',','')
a['Acciones']=a['Acciones'].str.replace('.00','')
a['Acciones']=a['Acciones'].astype(int)

In [ ]:
join = p_prices.join(acciones[0])